# Training Tip
- test set
- learning rate
- data preprocessing
- overfitting
- regularization

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

## Training and Testset

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [3]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

### Model -> softmax classifier

In [4]:
class Softmaxclassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)

    def forward(self,x):
        return self.linear(x)

In [5]:
model = Softmaxclassifier()

In [6]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(),lr = 0.1)

train 하는 함수

In [7]:
def train(model,optimizer,x_train,y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs + 1):

        # H(x) 계산
        prediction = model(x_train)
        # cost 계산
        cost = F.cross_entropy(prediction,y_train)
        # cost 로 h(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

test 하는 함수

In [8]:
def test(model,optimizer,x_test,y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction ,y_test)
    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [9]:
train(model,optimizer,x_train,y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365
Epoch   20/20 Cost: 0.983424


In [10]:
test(model,optimizer,x_test,y_test)

Accuracy: 0.0% Cost: 1.386150


## Learning Rate
Gradient Descent 에서의 $\alpha$ 값

optimizer = optim.SGD(model.parameters(), lr=0.1) 에서 lr=0.1 이다

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [11]:
model = Softmaxclassifier()
optimizer = optim.SGD(model.parameters(),lr = 1e5)

In [12]:
train(model,optimizer,x_train,y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.250000
Epoch    3/20 Cost: 1198378.875000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.250000
Epoch    7/20 Cost: 1532260.250000
Epoch    8/20 Cost: 1651503.750000
Epoch    9/20 Cost: 521878.562500
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.375000
Epoch   12/20 Cost: 918691.375000
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.125000
Epoch   16/20 Cost: 1140048.875000
Epoch   17/20 Cost: 940566.375000
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.625000
Epoch   20/20 Cost: 200090.921875


learning rate이 너무 작으면 cost가 거의 줄어들지 않는다

In [13]:
model = Softmaxclassifier()
optimizer = optim.SGD(model.parameters(),lr = 1e-10)
train(model,optimizer,x_train,y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324
Epoch   20/20 Cost: 3.187324


## Data Processing (데이터 전처리)
데이터를 zero-center 하고 normalize하기.-> 정규분포화

In [14]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$
여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [15]:
mu = x_train.mean(dim = 0)
print(mu)

tensor([84.8000, 84.6000, 85.6000])


In [16]:
sigma = x_train.std(dim = 0)
print(sigma)

tensor([11.0544, 12.2393, 12.6214])


In [17]:
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


학습한 성능 확인하기

In [18]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [19]:
model = MultivariateLinearRegressionModel()

In [20]:
optimizer = optim.SGD(model.parameters(),lr=1e-1)

In [21]:
def train(model,optimizer,x_train,y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs + 1):
        # H(x) 계산
        prediction = model(x_train)
        # Cost 계산
        cost = F.mse_loss(prediction,y_train)
        # Cost러 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [22]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29729.949219
Epoch    1/20 Cost: 18889.082031
Epoch    2/20 Cost: 12048.975586
Epoch    3/20 Cost: 7699.843750
Epoch    4/20 Cost: 4924.700684
Epoch    5/20 Cost: 3151.020264
Epoch    6/20 Cost: 2016.562866
Epoch    7/20 Cost: 1290.709106
Epoch    8/20 Cost: 826.216003
Epoch    9/20 Cost: 528.952271
Epoch   10/20 Cost: 338.703308
Epoch   11/20 Cost: 216.940063
Epoch   12/20 Cost: 139.006989
Epoch   13/20 Cost: 89.125130
Epoch   14/20 Cost: 57.196083
Epoch   15/20 Cost: 36.757317
Epoch   16/20 Cost: 23.672049
Epoch   17/20 Cost: 15.293401
Epoch   18/20 Cost: 9.927165
Epoch   19/20 Cost: 6.488903
Epoch   20/20 Cost: 4.284752


## Overfitting
너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.

이것을 방지하는 방법은 크게 세 가지인데:

1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. Regularization

In [23]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs + 1):

        # H(x) 계산
        prediction = model(x_train)
        # Cost 계산
        cost = F.mse_loss(prediction, y_train)
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
        cost += l2_reg
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))


In [24]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 29632.869141
Epoch    2/20 Cost: 18897.533203
Epoch    3/20 Cost: 12122.802734
Epoch    4/20 Cost: 7814.006836
Epoch    5/20 Cost: 5064.236816
Epoch    6/20 Cost: 3306.669678
Epoch    7/20 Cost: 2182.488770
Epoch    8/20 Cost: 1463.201904
Epoch    9/20 Cost: 1002.911377
Epoch   10/20 Cost: 708.338318
Epoch   11/20 Cost: 519.812500
Epoch   12/20 Cost: 399.154022
Epoch   13/20 Cost: 321.930054
Epoch   14/20 Cost: 272.503662
Epoch   15/20 Cost: 240.868073
Epoch   16/20 Cost: 220.618439
Epoch   17/20 Cost: 207.656235
Epoch   18/20 Cost: 199.358002
Epoch   19/20 Cost: 194.044785
Epoch   20/20 Cost: 190.642242
Epoch   21/20 Cost: 188.462555
